In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from wall_analysis import parse_calibration_experiments

fname = 'results/calibration.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    print('could not read', fname)
    df_total = parse_calibration_experiments()
    pd.to_pickle(df_total, fname)
    print('saved as', fname)

In [ ]:
print(df_total.appendix.unique())
df_total.loc[df_total.appendix=='_HALL3ok', 'appendix'] = '_HALL3'
print(df_total.appendix.unique())
df_total.tail()

In [ ]:
# calculate psd for sweep without snr selection
from wall_analysis import filter_by_dicts, extract_linear_psd

dict_chosen = [{
    'snr': 0,
    'motors': 0,
    'source': 'sweep',
}]

# found visually from spectrogram: 
sweep_lines = dict(
    slope=4000/280,
    offset=400,
    delta=20,
)

df_chosen = filter_by_dicts(df_total, dict_chosen)
for j, row in df_chosen.iterrows():
    spec = np.sum(np.abs(row.signals_f), axis=1) # sum over all mics

    fig, axs  = plt.subplots(1, 2)
    fig.set_size_inches(10, 5)
    fig.suptitle(f'experiment "{row.appendix}"')
    axs[0].pcolorfast(range(spec.shape[0]), row.frequencies, np.log10(spec.T))
    psd = extract_linear_psd(row.signals_f, row.frequencies, ax=axs[0], **sweep_lines)

    for i in range(psd.shape[0]):
        axs[1].semilogy(row.frequencies, psd[i], label=f'mic{i}')
    plt.legend(loc='upper left')

In [ ]:
from wall_analysis import extract_psd, extract_psd_dict
psd_dict1 = [{0: [], 100:[1, 2, 3], 200:[0, 4, 5]}, {100:[0, 1], 200:[1, 2]}]
psd_dict2 = [{100:[1, 2, 5], 250:[1, 4, 5]}, {100:[0, 1], 250:[3, 4]}]
psd_dict_list = [psd_dict1, psd_dict2]
psd, frequencies, psd_std = extract_psd(psd_dict_list)
print(psd.shape)
print(psd, frequencies)

In [ ]:
# calculate psd for sweep without snr selection
from wall_analysis import filter_by_dicts

dict_chosen = [{
    'snr': 1,
    'motors': 0,
}]

#from crazyflie_description_py.parameters import N_BUFFER, FS
#all_frequencies = list(np.round(np.fft.rfftfreq(N_BUFFER, 1/FS)))
#print(all_frequencies)

kwargs = {
    'sweep': {
        'min_t': 105,
        'max_t': 320
    },
    'sweep_buzzer': {
        'min_t': 430,
        'max_t': 1230
    }
}

# need this to assign list to this columns
df_total = df_total.assign(psd_dict=None)

df_chosen = filter_by_dicts(df_total, dict_chosen)
for (source, motors), df in df_chosen.groupby(['source', 'motors']):
    fig, ax  = plt.subplots()
    fig.set_size_inches(5, 5)
    ax.grid(which='both')
    legend = []
    for j, row in df.iterrows():
        psd_dict = extract_psd_dict(row.signals_f, row.frequencies_matrix, ax=ax, **kwargs[source])
        df_total.loc[j, 'psd_dict'] = psd_dict
        legend.append(row.appendix)

In [ ]:
kwargs_harmonics = {
    'sweep': {
        'min_t': 40,
        'max_t': 95 
    },
    'sweep_buzzer': {
        'min_t': 40,
        'max_t': 420 
    }
}

# need this to assign list to this columns
df_total = df_total.assign(psd_dict_harmonics=None)
df_chosen = filter_by_dicts(df_total, dict_chosen)
for source, df in df_chosen.groupby('source'):
    fig, ax  = plt.subplots()
    fig.set_size_inches(5, 5)
    ax.grid(which='both')
    for j, row in df.iterrows():
        psd_dict_harmonics = extract_psd_dict(row.signals_f, row.frequencies_matrix, ax=ax, **kwargs_harmonics[source])
        df_total.loc[j, 'psd_dict_harmonics'] = psd_dict_harmonics

In [ ]:
mic_idx = 0

df_chosen = filter_by_dicts(df_total, dict_chosen)
for source, df in df_chosen.groupby('source'):
    fig, axs = plt.subplots(len(df), sharey=True, sharex=True)
    fig.set_size_inches(10, 10)
    fig.suptitle(f'{source}')
    
    for j_idx, (j, row) in enumerate(df.iterrows()):
        axs[j_idx].set_title(f'experiment "{row.appendix}"')
        
        for key, vals in row.psd_dict[mic_idx].items():
            axs[j_idx].scatter([key]*len(vals), vals, c=range(len(vals)), cmap='inferno')
        for key, vals in row.psd_dict_harmonics[mic_idx].items():
            axs[j_idx].scatter([key]*len(vals), vals, c=range(len(vals)), cmap='cool')
        
        axs[j_idx].set_yscale('log')

In [ ]:
for source, df in df_chosen.groupby('source'):
    for j, row in df.iterrows():
        n_mics = row.signals_f.shape[1]
        
        psd, frequencies, psd_std = extract_psd([row.psd_dict], verbose=False) 
        
        fig, ax  = plt.subplots()
        fig.set_size_inches(10, 5)
        fig.suptitle(f'{source}: experiment "{row.appendix}"')
        for i in range(n_mics):
            mask = psd[i, :] > 0
            ax.plot(frequencies[mask], psd[i, mask], color=f'C{i}', label=f'mic{i}', marker='o')
        ax.set_yscale('log')
        ax.legend(loc='upper left')

In [ ]:
#chosen experiments for calibration, based on variance above.  
#chosen_experiments = ['_HALL', '_HALL2', '_HALL3']
chosen_experiments = ['_HALL', '_HALL2']

df_calib = pd.DataFrame(index=[], columns=['frequencies', 'psd', 'psd_std', 'type', 'source', 
                                           'method'])
methods = ['median', 'median-reject'] # 'mean', 'mean-reject'

for source, df in df_chosen.groupby('source'):
    for type_ in ['psd_dict', 'psd_dict_harmonics']:
        
        psd_dict_list = []
        for appendix in chosen_experiments:
            psd_dict = list(df.loc[df.appendix==appendix, type_].values)
            psd_dict_list += psd_dict
    
        for method in methods:
            psd, frequencies, psd_std = extract_psd(psd_dict_list, method=method, verbose=False) 
            df_calib.loc[len(df_calib), :] = {
                'frequencies': frequencies,
                'psd': psd, 
                'psd_std': psd_std, 
                'type': type_, 
                'source': source,
                'method': method
            }
    
fname = f'results/calibration_results.pkl'
pd.to_pickle(df_calib, fname)
print(f'saved as {fname}')
#df_chosen = df_total.loc[]

In [ ]:
psd_all = np.concatenate(df_calib.psd.values, axis=1)
freqs_all = np.concatenate(df_calib.frequencies.values)
y_min = np.min(psd_all)
y_max = np.max(psd_all)
x_min = np.min(freqs_all) 
x_max = np.max(freqs_all) 

for i, row in df_calib.iterrows():
    fig, ax = plt.subplots()
    fig.set_size_inches(7, 5)
    ax.set_title(f'{row.source}{row.type.replace("psd_dict", "")}, {row.method}')
    for i in range(row.psd.shape[0]):
        ax.errorbar(x=row.frequencies, y=row.psd[i, :], yerr=row.psd_std[i, :])
    ax.set_yscale('log')
    ax.set_ylim(y_min, y_max)
    ax.set_xlim(x_min, x_max)

# frequency selection with motors on

In [ ]:
chosen_dict = dict(
    #appendix='_BC329',
    appendix='',
    motors='all43000',
    snr=1,
    source='sweep'
)
df_chosen = filter_by_dicts(df_total, [chosen_dict])
assert len(df_chosen) == 1
row = df_chosen.iloc[0]

fig = plt.figure() 
fig.set_size_inches(10, 5)
plt.title(f'strongest frequency')
plt.plot(row.seconds, row.frequencies_matrix[:, 0])
plt.ylim(1000, 5000)

n_freqs = 32
cmap = plt.get_cmap('inferno', n_freqs)
fig = plt.figure()
plt.title(f'strongest {n_freqs} frequencies (dark to light)')
fig.set_size_inches(10, 5)
for i in range(n_freqs):
    color = i / n_freqs #(n_freqs - i)/n_freqs
    plt.scatter(row.seconds, row.frequencies_matrix[:, i], color=cmap(color), s=2)
plt.ylim(1000, 5000)

In [ ]:
chosen_dict = dict(
    #appendix='_BC329',
    #appendix='',
    motors='all43000',
    snr=0,
    source='sweep'
)
df_chosen = filter_by_dicts(df_total, [chosen_dict])
for i, row in df_chosen.iterrows():
    fig, ax = plt.subplots()
    spec = np.sum(np.abs(row.signals_f), axis=1)
    ax.pcolorfast(row.seconds, row.frequencies, np.log10(spec[1:, 1:].T))

In [ ]:
chosen_dict = dict(
    motors='all43000',
    #snr=1,
    source='sweep'
)
mic_idx = 0
df_chosen = filter_by_dicts(df_total, [chosen_dict])

plt.figure()
for i, row in df_chosen.iterrows():
    if row.frequencies_matrix is None:
        frequencies_matrix = np.repeat(row.frequencies[None, :], len(row.seconds), axis=0)
    else:
        frequencies_matrix = row.frequencies_matrix
    print(frequencies_matrix.shape)
    psd_dict = extract_psd_dict(row.signals_f, frequencies_matrix, n_freq=32)
    psd, frequencies, psd_std = extract_psd([psd_dict])
    
    plt.plot(frequencies, psd[mic_idx], marker='o', label=f'snr: {row.snr}')
plt.yscale('log')
plt.legend()